In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

tf.random.set_seed(777)

2.17.0


In [3]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

In [4]:
nb_classes = 3

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [5]:
# W와 b 세팅
W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]

print(W, b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.7706481 ,  0.37335402, -0.05576323],
       [ 0.00358377, -0.5898363 ,  1.5702795 ],
       [ 0.2460895 , -0.09918973,  1.4418385 ],
       [ 0.3200988 ,  0.526784  , -0.7703731 ]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-1.3080608 , -0.13253094,  0.5513761 ], dtype=float32)>


In [6]:
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis(x_data))

tf.Tensor(
[[1.36571955e-02 7.90162291e-03 9.78441179e-01]
 [3.92597765e-02 1.70347411e-02 9.43705440e-01]
 [3.80385250e-01 1.67723149e-01 4.51891541e-01]
 [3.23390484e-01 5.90759404e-02 6.17533624e-01]
 [3.62997366e-06 6.20727221e-08 9.99996245e-01]
 [2.62520202e-02 1.07279625e-02 9.63019967e-01]
 [1.56525093e-05 4.21802724e-07 9.99983847e-01]
 [2.94076904e-06 3.81133241e-08 9.99996960e-01]], shape=(8, 3), dtype=float32)


In [7]:
# softmax one-hot test
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(hypothesis(sample_db))

tf.Tensor([[0.9302204  0.06200533 0.00777428]], shape=(1, 3), dtype=float32)


In [8]:
def cost_fn(X, Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.math.log(logits), axis = 1)
    cost_mean = tf.reduce_mean(cost)

    return cost_mean

print(cost_fn(x_data, y_data))

tf.Tensor(6.07932, shape=(), dtype=float32)


In [9]:
x = tf.constant(3.0)

with tf.GradientTape() as g:
    g.watch(x)
    y = x * x

dy_dx = g.gradient(y, x)
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [11]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)

        return grads

print(grad_fn(x_data, y_data))

[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0.06914607, -0.6509784 ,  0.5818323 ],
       [-1.5221257 , -1.214863  ,  2.7369885 ],
       [-1.2473829 , -1.7611003 ,  3.0084827 ],
       [-1.2014607 , -1.8659232 ,  3.0673835 ]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.15212914, -0.342192  ,  0.49432108], dtype=float32)>]


In [12]:
def fit(X, Y, epochs=2000, verbose=100):
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))

fit(x_data, y_data)

Loss at epoch 1: 2.849417
Loss at epoch 100: 0.684151
Loss at epoch 200: 0.613813
Loss at epoch 300: 0.558204
Loss at epoch 400: 0.508306
Loss at epoch 500: 0.461058
Loss at epoch 600: 0.415072
Loss at epoch 700: 0.369636
Loss at epoch 800: 0.324533
Loss at epoch 900: 0.280721
Loss at epoch 1000: 0.246752
Loss at epoch 1100: 0.232798
Loss at epoch 1200: 0.221645
Loss at epoch 1300: 0.211476
Loss at epoch 1400: 0.202164
Loss at epoch 1500: 0.193606
Loss at epoch 1600: 0.185714
Loss at epoch 1700: 0.178415
Loss at epoch 1800: 0.171645
Loss at epoch 1900: 0.165351
Loss at epoch 2000: 0.159483


In [21]:
# 정확도 체크
sample_data = [[2,1,3,2]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis(sample_data)

print(a)
print(tf.argmax(a, 1))

tf.Tensor(
[[0.23090686 0.7200088  0.04908434]
 [0.2187771  0.7384027  0.04282029]
 [0.20615774 0.7566084  0.03723383]
 [0.11932245 0.8698279  0.01084957]
 [0.16614637 0.8237168  0.01013677]
 [0.1334393  0.85745764 0.009103  ]
 [0.33841035 0.6489778  0.01261187]
 [0.42188144 0.5649803  0.01313805]], shape=(8, 3), dtype=float32)
tf.Tensor([1 1 1 1 1 1 1 1], shape=(8,), dtype=int64)


In [14]:
b = hypothesis(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1))

tf.Tensor(
[[2.1976039e-06 1.2331199e-03 9.9876475e-01]
 [1.1288594e-03 8.1546724e-02 9.1732436e-01]
 [2.2205660e-07 1.6418649e-01 8.3581328e-01]
 [6.3921934e-06 8.5045439e-01 1.4953916e-01]
 [2.6150793e-01 7.2644734e-01 1.2044546e-02]
 [1.3783264e-01 8.6213988e-01 2.7417602e-05]
 [7.4242103e-01 2.5754192e-01 3.6978636e-05]
 [9.2197543e-01 7.8024052e-02 6.0005920e-07]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
